In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import collections
import re
import pprint as pp
import numpy as np
import matplotlib.pyplot as plt

import math
import gzip
import pickle as pkl
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime
import random
random.seed(123)
import os
import time
import tflearn

import multiprocessing as mp
from multiprocessing.pool import ThreadPool
from multiprocessing import Process

from keras import Sequential
from keras.layers import Dense,Dropout,BatchNormalization, LeakyReLU
from keras import optimizers
from keras import losses

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer 

import tensorflow as tf

import itertools
import gzip
import pickle as pkl

import fonctions
import batch_processing
import metric_loading

from os import listdir
from tqdm.notebook import tqdm
from os.path import isfile, join

from scipy.sparse import hstack
from keras import backend as K
from keras import callbacks
from sklearn.metrics import log_loss
from tensorflow import ones_like, equal, log
from keras import backend as K
from tensorflow.python import mul
import tensorflow as tf

Instructions for updating:
Colocations handled automatically by placer.


Using TensorFlow backend.


In [9]:
def read_batch(directory, doc_name):
    
    df = pd.read_csv(directory+doc_name, encoding="utf-8", sep=',', header=0, index_col=False,low_memory=False)
    df = df.drop(df.columns[ [0] ], axis=1)
    df.index = df.tweet_id+'-'+df.engaging_user_id
    
    labels= ['like_timestamp','retweet_timestamp','retweet_with_comment_timestamp','reply_timestamp']    
     
    X = df.drop( ['mean_daily_author_presence','std_daily_author_presence',
                  'language','tweet_id','tweet_timestamp','tweet_utctimestamp',
                  'engaged_with_user_account_creation', 'text_tokens',
                  'present_links','present_domains','hashtags',
                  'engaging_user_account_creation', 'present_media',
                  'engaged_with_user_id', 'engaging_user_id' ],axis=1)
    
    X = X[ labels+['engaged_with_user_follower_count','engaged_with_user_following_count',
    'engaged_with_user_is_verified','engaging_user_follower_count','engaging_user_following_count',
    'engaging_user_is_verified','engagee_follows_engager',
    'hour_ 00','hour_ 01','hour_ 02','hour_ 03','hour_ 04',
    'hour_ 05','hour_ 06','hour_ 07','hour_ 08','hour_ 09',
    'hour_ 10','hour_ 11','hour_ 12','hour_ 13','hour_ 14',
    'hour_ 15','hour_ 16','hour_ 17','hour_ 18','hour_ 19','hour_ 20',
    'hour_ 21','hour_ 22','hour_ 23','day_0','day_1','day_2','day_3', 'day_4','day_5', 'day_6',
    'user_account_age','user_presence','author_account_age',
    'author_presence','tweet_type_Quote','tweet_type_Retweet','tweet_type_TopLevel',
    'photo','video','gif','nb_hashtags','nb_links','nb_domains','len_tweet',
    'taste_rt_author','taste_rtc_author',
    'taste_rpl_author','taste_lk_author','taste_rt_hashtag',
    'taste_rtc_hashtag','taste_rpl_hashtag','taste_lk_hashtag',
    'content_link','content_text','content_photo_video','content_gif',
    'user_0','user_1','user_2','user_3',
    'user_4','user_5','user_6','user_7',
    'user_8','user_9','user_10','user_11' ] ]
    
    X = shuffle(X, random_state=0)
    
    nan_cols = [i for i in X.columns if X[i].isnull().any() ]

    #########################################################
    ###### no a priori model
    
    X1 = X[  ~(X[ nan_cols ].notnull().all(1) ) ]
    y1= X1.filter( labels, axis=1 )
    X1 = X1.drop(labels,axis=1)
    
    ##########################################################
    ###### a priori model
    
    X2 = X[  X[ nan_cols ].notnull().all(1) ]
    y2 = X2.filter(labels, axis=1)
    X2 = X2.drop(labels,axis=1)

    return X1, y1, X2, y2

def save_model(model_id, classifier):
    
    with gzip.open('/home/maxime/Desktop/RecSys2020/modeles/model_multilabel_NN_{}.pkl.gz'.format(model_id),'wb') as f:
        pkl.dump(classifier,f)
        
    return True

def build_multilabel_NN(nb):
    
    classifier = Sequential()
    classifier.add(Dense( nb+20, input_dim=nb ,activation='relu' ))
    classifier.add(Dropout(0.3))
    classifier.add(Dense( nb,activation='relu' ))
    classifier.add(Dense( nb-15,activation='relu' ))
    classifier.add(Dense( nb-30,activation='relu' ))
    classifier.add(Dense( 15,activation='relu' ))
    classifier.add(Dense(4, activation='sigmoid'))
    opt = optimizers.Adam()
    classifier.compile(optimizer = opt, loss=['binary_crossentropy'] ) 
    
    return classifier

def custom_loss(y_true,y_pred):
    
    substr = y_pred[:,1] - y_pred[:,2]
    min_value = K.constant(-1, substr.dtype.base_dtype)
    zeros = K.constant(0, substr.dtype.base_dtype)
    x = K.clip(substr, min_value, zeros)
   
    bce = tf.keras.losses.BinaryCrossentropy()
    
    total_loss =  bce(y_true, y_pred) +  100000 * K.sum(  K.square(x) ) 
    
    return total_loss

def train_on_chunk(chunk, model_id, classifier1,classifier2, nb_epochs):

    directory = '/home/maxime/Desktop/RecSys2020/data/processed_batches1/'
    sc = StandardScaler()

    for _ in range(nb_epochs):

        iteration=1
        for batch_file in chunk:

            print(iteration)
            X1, y1, X2, y2 = read_batch(directory, batch_file)
            
            #########################################################
            ###### train no a priori model
            
            X1 = X1.drop(['user_0', 'user_1', 'user_2', 'user_3', 'user_4', 'user_5', 'user_6',
                          'user_7', 'user_8', 'user_9', 'user_10', 'user_11',
                          'content_link', 'content_text', 'content_photo_video', 'content_gif',
                          'taste_rt_author','taste_rtc_author','taste_rpl_author','taste_lk_author',
                          'taste_rt_hashtag','taste_rtc_hashtag','taste_rpl_hashtag','taste_lk_hashtag'],axis=1)
            
            X1 = sc.fit_transform(X1) 
            classifier1.fit(X1, y1, verbose=1 )
    
            ##########################################################
            ###### train a priori model
        
            X2 = sc.fit_transform(X2) 
            classifier2.fit(X2, y2, verbose=1 )
            
            iteration=iteration+1
            
    save_model(model_id, classifier1)
    save_model(str(model_id)+'_wpriori', classifier2)

    return classifier1,classifier2

In [10]:
%%time

batch_path='/home/maxime/Desktop/RecSys2020/data/processed_batches1/'
chunk = [f for f in listdir(batch_path) if isfile(join(batch_path, f))]

model_id=3005

classifier1 = build_multilabel_NN(52)
classifier2 = build_multilabel_NN(76)

classifier1, classifier2 = train_on_chunk(chunk[:500], model_id, classifier1,classifier2, 1)

1
Epoch 1/1
21073/21073 [==============================] - 1s 62us/step - loss: 0.2746
Epoch 1/1
8927/8927 [==============================] - 1s 113us/step - loss: 0.2759
2
Epoch 1/1
21144/21144 [==============================] - 2s 80us/step - loss: 0.2540
Epoch 1/1
8856/8856 [==============================] - 1s 122us/step - loss: 0.2317
3
Epoch 1/1
21218/21218 [==============================] - 1s 54us/step - loss: 0.2511
Epoch 1/1
8782/8782 [==============================] - 1s 66us/step - loss: 0.2167
4
Epoch 1/1
20991/20991 [==============================] - 1s 49us/step - loss: 0.2490
Epoch 1/1
9009/9009 [==============================] - 1s 62us/step - loss: 0.2174
5
Epoch 1/1
21392/21392 [==============================] - 1s 49us/step - loss: 0.2497
Epoch 1/1
8608/8608 [==============================] - 0s 56us/step - loss: 0.2099
6
Epoch 1/1
20717/20717 [==============================] - 1s 48us/step - loss: 0.2464
Epoch 1/1
9283/9283 [==============================] - 1s 56u

21299/21299 [==============================] - 1s 40us/step - loss: 0.2373
Epoch 1/1
8701/8701 [==============================] - 0s 44us/step - loss: 0.1948
97
Epoch 1/1
21595/21595 [==============================] - 1s 47us/step - loss: 0.2421
Epoch 1/1
8405/8405 [==============================] - 1s 62us/step - loss: 0.2011
98
Epoch 1/1
20825/20825 [==============================] - 1s 43us/step - loss: 0.2374
Epoch 1/1
9175/9175 [==============================] - 1s 57us/step - loss: 0.2009
99
Epoch 1/1
21052/21052 [==============================] - 1s 42us/step - loss: 0.2429
Epoch 1/1
8948/8948 [==============================] - 0s 48us/step - loss: 0.1991
100
Epoch 1/1
21055/21055 [==============================] - 1s 43us/step - loss: 0.2408
Epoch 1/1
8945/8945 [==============================] - 0s 45us/step - loss: 0.1946
101
Epoch 1/1
20995/20995 [==============================] - 1s 39us/step - loss: 0.2413
Epoch 1/1
9005/9005 [==============================] - 0s 44us/step 

KeyboardInterrupt: 

In [6]:
def read_val_batch(directory, doc_name):
    
    df = pd.read_csv(directory+doc_name, encoding="utf-8", sep=',', header=0, index_col=False,low_memory=False)
    df = df.drop(df.columns[ [0] ], axis=1)
    df.index = df.tweet_id+'-'+df.engaging_user_id
    
    labels= ['like_timestamp','retweet_timestamp','retweet_with_comment_timestamp','reply_timestamp']
    
    X = df.drop( ['mean_daily_author_presence','std_daily_author_presence',
                  'language','tweet_id','tweet_timestamp','tweet_utctimestamp',
                  'engaged_with_user_account_creation', 'text_tokens',
                  'present_links','present_domains','hashtags',
                  'engaging_user_account_creation', 'present_media',
                  'engaged_with_user_id', 'engaging_user_id' ],axis=1)
    
    X = X[ labels+['engaged_with_user_follower_count','engaged_with_user_following_count',
    'engaged_with_user_is_verified','engaging_user_follower_count','engaging_user_following_count',
    'engaging_user_is_verified','engagee_follows_engager',
    'hour_ 00','hour_ 01','hour_ 02','hour_ 03','hour_ 04',
    'hour_ 05','hour_ 06','hour_ 07','hour_ 08','hour_ 09',
    'hour_ 10','hour_ 11','hour_ 12','hour_ 13','hour_ 14',
    'hour_ 15','hour_ 16','hour_ 17','hour_ 18','hour_ 19','hour_ 20',
    'hour_ 21','hour_ 22','hour_ 23','day_0','day_1','day_2','day_3', 'day_4','day_5', 'day_6',
    'user_account_age','user_presence','author_account_age',
    'author_presence','tweet_type_Quote','tweet_type_Retweet','tweet_type_TopLevel',
    'photo','video','gif','nb_hashtags','nb_links','nb_domains','len_tweet',
    'taste_rt_author','taste_rtc_author',
    'taste_rpl_author','taste_lk_author','taste_rt_hashtag',
    'taste_rtc_hashtag','taste_rpl_hashtag','taste_lk_hashtag',
    'content_link','content_text','content_photo_video','content_gif',
    'user_0','user_1','user_2','user_3',
    'user_4','user_5','user_6','user_7',
    'user_8','user_9','user_10','user_11' ] ]
    
    X = shuffle(X, random_state=0)
    
    nan_cols = [i for i in X.columns if X[i].isnull().any() ]

    
    #########################################################
    ###### no a priori model
    
    X1 = X[  ~(X[ nan_cols ].notnull().all(1) ) ]
    y1 = X1.filter(labels, axis=1)
    X1.drop(labels,axis=1)
    
    ##########################################################
    ###### a priori model
    
    X2 = X[  X[ nan_cols ].notnull().all(1) ]
    y2 = X2.filter(labels, axis=1)
    X2.drop(labels,axis=1)

    return X1,y1,X2,y2

def validation_on_chunk(model_id, chunk, chunk_id,classifier1,classifier2):
    
    engagements = ['like_timestamp','retweet_timestamp','retweet_with_comment_timestamp','reply_timestamp']

    sc = StandardScaler()
    directory = '/home/maxime/Desktop/RecSys2020/data/val_pr_batches/'
    iteration=1
    
    for test_batch in chunk:
        print(str(iteration)+' '+test_batch)
        
        X1, y1, X2, y2 = read_batch(directory, test_batch)

        ##########################################################
        ###### train no a priori model
        
        X1 = X1.drop(['user_0', 'user_1', 'user_2', 'user_3', 'user_4', 'user_5', 'user_6',
                      'user_7', 'user_8', 'user_9', 'user_10', 'user_11',
                      'content_link', 'content_text', 'content_photo_video', 'content_gif',
                      'taste_rt_author','taste_rtc_author','taste_rpl_author','taste_rt_hashtag',
                      'taste_rtc_hashtag', 'taste_rpl_hashtag', 'taste_lk_author','taste_lk_hashtag'  ],axis=1)
   
        print(len(X1.index))
        index1 = X1.index
        X1 = sc.fit_transform(X1) 
        y_pred1 = classifier1.predict(X1, verbose=1) 
        y_pred1 = y_pred1.astype('float64')
        
        d1 = {'Tweet_Id': [ x.split('-')[0] for x in index1 ], 'User_Id': [ x.split('-')[1] for x in index1 ],
              'like_timestamp':y_pred1[:,0],'retweet_timestamp':y_pred1[:,1],
              'retweet_with_comment_timestamp':y_pred1[:,2],'reply_timestamp':y_pred1[:,3],
              'truth_like_timestamp':y1.like_timestamp,'truth_retweet_timestamp':y1.retweet_timestamp,
              'truth_retweet_with_comment_timestamp':y1.retweet_with_comment_timestamp,'truth_reply_timestamp':y1.reply_timestamp
             }


        result1 = pd.DataFrame(data=d1)
        
        #########################################################
        ###### train a priori model
        
        print(len(X2.index))
        index2 = X2.index
        X2 = sc.fit_transform(X2) 
        y_pred2 = classifier2.predict(X2) 
        y_pred2 =  y_pred2.astype('float64')
        
        d2 = {'Tweet_Id': [ x.split('-')[0] for x in index2 ], 'User_Id': [ x.split('-')[1] for x in index2 ],
              'like_timestamp':y_pred2[:,0],'retweet_timestamp':y_pred2[:,1],
              'retweet_with_comment_timestamp':y_pred2[:,2],'reply_timestamp':y_pred2[:,3],
             'truth_like_timestamp':y2.like_timestamp,'truth_retweet_timestamp':y2.retweet_timestamp,
               'truth_retweet_with_comment_timestamp':y2.retweet_with_comment_timestamp,'truth_reply_timestamp':y2.reply_timestamp }

        result2 = pd.DataFrame(data=d2)

        resultat = pd.concat([result1,result2], axis=0)

        for idx, engagement in enumerate(engagements):
            result = resultat.filter(['Tweet_Id','User_Id', engagement,'truth_'+str(engagement) ],axis=1) 
            result.columns = ['Tweet_Id','User_Id', 'Prediction', 'Truth'] 
            
            result.to_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_{}_{}.csv'.format(engagement, model_id),
            header=None, index=None, sep=',', mode='a')
        
        iteration=iteration+1
  
    return True

In [7]:
eval_batch_path='/home/maxime/Desktop/RecSys2020/data/val_pr_batches'
eval_batch_list = [f for f in listdir(eval_batch_path) if isfile(join(eval_batch_path, f))]
model_id = 3005

validation_on_chunk(model_id, eval_batch_list, 0,classifier1,classifier2)

1 pr_job_993189.tsv
22078
22078/22078 [==============================] - 0s 14us/step
7922
2 pr_job_993485.tsv
21998
21998/21998 [==============================] - 0s 12us/step
8002
3 pr_job_993557.tsv
22283
22283/22283 [==============================] - 0s 12us/step
7717
4 pr_job_993213.tsv
21789
21789/21789 [==============================] - 0s 12us/step
8211
5 pr_job_993144.tsv
22018
22018/22018 [==============================] - 0s 12us/step
7982
6 pr_job_993447.tsv
21778
21778/21778 [==============================] - 0s 12us/step
8222
7 pr_job_993545.tsv
22250
22250/22250 [==============================] - 0s 11us/step
7750
8 pr_job_993400.tsv
21728
21728/21728 [==============================] - 0s 11us/step
8272
9 pr_job_993406.tsv
21779
21779/21779 [==============================] - 0s 13us/step
8221
10 pr_job_993357.tsv
22062
22062/22062 [==============================] - 0s 13us/step
7938
11 pr_job_993515.tsv
22163
22163/22163 [==============================] - 0s 12us/step
78

22123
22123/22123 [==============================] - 0s 11us/step
7877
91 pr_job_993312.tsv
22177
22177/22177 [==============================] - 0s 11us/step
7823
92 pr_job_993473.tsv
22161
22161/22161 [==============================] - 0s 11us/step
7839
93 pr_job_993477.tsv
22094
22094/22094 [==============================] - 0s 11us/step
7906
94 pr_job_993488.tsv
22099
22099/22099 [==============================] - 0s 11us/step
7901
95 pr_job_993355.tsv
22178
22178/22178 [==============================] - 0s 11us/step
7822
96 pr_job_993192.tsv
22133
22133/22133 [==============================] - 0s 11us/step
7867
97 pr_job_993242.tsv
21707
21707/21707 [==============================] - 0s 11us/step
8293
98 pr_job_993280.tsv
21788
21788/21788 [==============================] - 0s 11us/step
8212
99 pr_job_993275.tsv
21724
21724/21724 [==============================] - 0s 12us/step
8276
100 pr_job_993432.tsv
22017
22017/22017 [==============================] - 0s 12us/step
7983
101 pr_j

22151/22151 [==============================] - 0s 11us/step
7849
267 pr_job_993507.tsv
21980
21980/21980 [==============================] - 0s 10us/step
8020
268 pr_job_993288.tsv
22222
22222/22222 [==============================] - 0s 11us/step
7778
269 pr_job_993453.tsv
21760
21760/21760 [==============================] - 0s 11us/step
8240
270 pr_job_993496.tsv
22258
22258/22258 [==============================] - 0s 11us/step
7742
271 pr_job_993503.tsv
22137
22137/22137 [==============================] - 0s 12us/step
7863
272 pr_job_993176.tsv
22022
22022/22022 [==============================] - 0s 12us/step
7978
273 pr_job_993529.tsv
22007
22007/22007 [==============================] - 0s 11us/step
7993
274 pr_job_993148.tsv
21795
21795/21795 [==============================] - 0s 11us/step
8205
275 pr_job_993544.tsv
22176
22176/22176 [==============================] - 0s 12us/step
7824
276 pr_job_993505.tsv
22079
22079/22079 [==============================] - 0s 11us/step
7921
277 p

21990/21990 [==============================] - 0s 11us/step
8010
443 pr_job_993202.tsv
22018
22018/22018 [==============================] - 0s 12us/step
7982
444 pr_job_993324.tsv
21988
21988/21988 [==============================] - 0s 12us/step
8012
445 pr_job_993132.tsv
22078
22078/22078 [==============================] - 0s 11us/step
7922
446 pr_job_993427.tsv
21647
21647/21647 [==============================] - 0s 11us/step
8353
447 pr_job_993291.tsv
22226
22226/22226 [==============================] - 0s 11us/step
7774
448 pr_job_993522.tsv
22013
22013/22013 [==============================] - 0s 12us/step
7987
449 pr_job_993389.tsv
21840
21840/21840 [==============================] - 0s 11us/step
8160
450 pr_job_993125.tsv
22038
22038/22038 [==============================] - 0s 11us/step
7962


True